# LangChain Expression Language (LCEL)

In [2]:
#!pip install pydantic==1.10.8

In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOllama
from langchain.schema.output_parser import StrOutputParser

## Simple Chain

In [2]:
import ollama

ollama.list()

{'models': [{'name': 'phi3:instruct',
   'model': 'phi3:instruct',
   'modified_at': '2024-06-02T11:16:07.7269586+05:30',
   'size': 2393232963,
   'digest': '64c1188f2485448235b2d371639a127fc0e4dc2cd3c041152368883c42eb2bd1',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'phi3',
    'families': ['phi3'],
    'parameter_size': '3.8B',
    'quantization_level': 'Q4_K_M'},
   'expires_at': '0001-01-01T00:00:00Z'},
  {'name': 'zephyr:latest',
   'model': 'zephyr:latest',
   'modified_at': '2024-05-04T17:03:40.8751826+05:30',
   'size': 4109854934,
   'digest': 'bbe38b81adec6be8ff951d148864ed15a368aa2e8534a5092d444f184a56e354',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '7B',
    'quantization_level': 'Q4_0'},
   'expires_at': '0001-01-01T00:00:00Z'}]}

In [3]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOllama(model='phi3:instruct')
output_parser = StrOutputParser()

In [4]:
chain = prompt | model | output_parser

In [5]:
chain.invoke({"topic": "bears"})

" Why don't bears use computers? Because they can't bear the thought of clicking!"

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [6]:
# from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [7]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma(collection_name="my_collection", embedding_function=embedding_function)

texts = ["harrison worked at kensho", "bears like to eat honey"]

db.add_texts(texts)
retriever = db.as_retriever()

c:\Users\kaila\DLAI\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
retriever.get_relevant_documents("where did harrison work?",)

c:\Users\kaila\DLAI\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [9]:
retriever.get_relevant_documents("what do bears like to eat")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [10]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [11]:
from langchain.schema.runnable import RunnableMap

In [12]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [13]:
chain.invoke({"question": "where did harrison work?"})

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


' Harrison worked at Kensho.'

## Fallbacks

In [14]:
import json

simple_model = ChatOllama(model="phi3:instruct")

simple_chain = simple_model | json.loads

In [15]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [16]:
simple_model.invoke(challenge)

AIMessage(content=' {\n  "poems": [\n    {\n      "title": "Ode to the West Wind",\n      "author": "Percy Bysshe Shelley",\n      "first_line": "Oh powerful spirit veiled in night!"\n    },\n    {\n      "title": "The Raven",\n      "author": "Edgar Allan Poe",\n      "first_line": "Once upon a midnight dreary, while I pondered, weak and weary"\n    },\n    {\n      "title": "Stopping by Woods on a Snowy Evening",\n      "author": "Robert Frost",\n      "first_line": "Whose woods these are I think I know."\n    }\n  ]\n}', response_metadata={'model': 'phi3:instruct', 'created_at': '2024-06-02T05:50:00.2674901Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 19372149700, 'load_duration': 1581700, 'prompt_eval_count': 27, 'prompt_eval_duration': 903064000, 'eval_count': 173, 'eval_duration': 18461832000}, id='run-60ae7172-9068-4017-a206-871179b77939-0')

**Note**: The next line is expected to fail.

In [17]:
try:
    simple_chain.invoke(challenge)
finally:
    print("yep")

yep


TypeError: the JSON object must be str, bytes or bytearray, not AIMessage

In [19]:
final_chain = simple_chain.with_fallbacks([chain])
# final_chain.invoke(challenge)

## Interface

In [20]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOllama(model="phi3:instruct")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [21]:
chain.invoke({"topic": "bears"})

" Why don't bears play poker? Too many bear hugs!\n\n(Note: This joke is intended for light-hearted entertainment and does not promote any form of animal mistreatment.)"

In [22]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

[' Why don\'t bears use computers?\n\nBecause they can\'t type "bear-ly"!\n\n(Note: This is intended as light humor and doesn\'t intend to mock any animal.)',
 ' Why did the frog say "blastoff" instead of "croak"? Because he was excited to be in space!\n\n(Note: This joke is light-hearted and doesn\'t mock real scientific endeavors involving amphibians.)']

In [ ]:
for t in chain.stream({"topic": "bears"}):
    print(t)

What
 do
 you
 call
 a
 bear
 that
'
s
 always
 hungry
?



A
 bear
-
licious
 bear
!



In [23]:
response = await chain.ainvoke({"topic": "bears"})
response

" Why don't bears like to go on dates?\n\nBecause they can't stand the thought of honey and ice cream!"

## Binding functions

In [ ]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOllama(model="phi3:instruct",temperature=0).bind(functions=functions)

In [60]:
runnable = prompt | model

# runnable.invoke({"input": "what is the weather in sf"})

In [3]:
from langchain_core.runnables import RunnableLambda

def add_one(x: int) -> int:
    return x + 1

def double(y: int) -> int:
    return y * 2

# Create RunnableLambdas
add_one_runnable = RunnableLambda(add_one)
double_runnable = RunnableLambda(double)

# Combine Runnables using the pipe operator
combined_runnable = add_one_runnable | double_runnable

# Execute the combined Runnable
result = combined_runnable.invoke(5) 
result

12